In [1]:
import asyncio
import gzip
import json
import logging
import os
import sys
import urllib
import warnings
from datetime import datetime

import dotenv
import pandas as pd
import requests
from sqlalchemy import create_engine, text

### Smoke Test For Model Health Check Point

In [3]:
import importlib.util

path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_model.py"
spec = importlib.util.spec_from_file_location("rerun_model", path)
rerun_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(rerun_model)

print("Model version:", rerun_model.get_model_version())


Model version: 16.15.6
